In [1]:
!pip install bitsandbytes
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM , BitsAndBytesConfig
import bitsandbytes as bnb
from torch.utils.data import DataLoader,Dataset
import torch
from tqdm import tqdm
from datasets import load_dataset
import os
from huggingface_hub import login

os.environ['HF_TOKEN']="HF_TOKEN_HERE"
login(token=os.environ["HF_TOKEN"])



Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


# Dataset

In [3]:
cp="neil-code/dialogsum-test"
ds=load_dataset(cp)

README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1999 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/499 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/499 [00:00<?, ? examples/s]

In [4]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1999
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 499
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 499
    })
})

In [5]:
ds['train'][0]

{'id': 'train_0',
 'dialogue': "#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.\n#Person2#: Ok, thanks doctor.",
 'summary': "Mr. Smith'

In [6]:
print(ds['train']['dialogue'][0])

#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?
#Person2#: I found it would be a good idea to get a check-up.
#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.
#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?
#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.
#Person2#: Ok.
#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?
#Person2#: Yes.
#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.
#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.
#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.
#Person2#: Ok, thanks doctor.


In [7]:
def add_instruction(dct):
    return {"input": f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
    
### Instruct: Summarise the following Conversation.

{dct['dialogue']}

### Output:
"""}

In [8]:
ds=ds.map(add_instruction)

Map:   0%|          | 0/1999 [00:00<?, ? examples/s]

Map:   0%|          | 0/499 [00:00<?, ? examples/s]

Map:   0%|          | 0/499 [00:00<?, ? examples/s]

In [9]:
ds['train']

Dataset({
    features: ['id', 'dialogue', 'summary', 'topic', 'input'],
    num_rows: 1999
})

In [10]:
ds=ds.remove_columns(['dialogue','id','topic'])

In [11]:
print(ds['train'][0]['input'])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
    
### Instruct: Summarise the following Conversation.

#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?
#Person2#: I found it would be a good idea to get a check-up.
#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.
#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?
#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.
#Person2#: Ok.
#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?
#Person2#: Yes.
#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.
#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.
#P

In [12]:
len(ds['train'])

1999

In [13]:
train_dataset=ds['train']
test_dataset=ds['test']
val_dataset=ds['validation']

In [14]:
lst=[]
for i in train_dataset:
    lst.append(i['input'])

In [15]:
len(lst),len(set(lst))

(1999, 1999)

In [16]:
lst=[]
for i in test_dataset:
    lst.append(i['input'])

In [17]:
len(lst),len(set(lst))

(499, 167)

In [18]:
lst=[]
for i in val_dataset:
    lst.append(i['input'])

In [19]:
len(lst),len(set(lst))

(499, 499)

In [20]:
## test dataset contains a lot of duplicates so using val dataset as test dataset
test_dataset=val_dataset

In [21]:
len(test_dataset)

499

In [22]:
device='cuda' if torch.cuda.is_available() else 'cpu'

In [23]:
cp="microsoft/phi-2"
tokenizer=AutoTokenizer.from_pretrained(cp,padding_side='left')

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [24]:
def collate_fn(batch):
    questions=tokenizer([i['input'] for i in batch],return_tensors='pt',padding=True,truncation=True)
    summaries=[i['summary'] for i in batch]
    return {'inputs':questions,'summaries':summaries}

In [25]:
tokenizer.pad_token=tokenizer.eos_token

In [26]:
batch_size=8
test_data=DataLoader(test_dataset,batch_size=batch_size,collate_fn=collate_fn)

In [27]:
batch1=next(iter(test_data))

In [28]:
batch1

{'inputs': {'input_ids': tensor([[50256, 50256, 50256,  ..., 25235,    25,   198],
         [50256, 50256, 50256,  ..., 25235,    25,   198],
         [50256, 50256, 50256,  ..., 25235,    25,   198],
         ...,
         [50256, 50256, 50256,  ..., 25235,    25,   198],
         [50256, 50256, 50256,  ..., 25235,    25,   198],
         [21106,   318,   281,  ..., 25235,    25,   198]]), 'attention_mask': tensor([[0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1],
         ...,
         [0, 0, 0,  ..., 1, 1, 1],
         [0, 0, 0,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]])},
 'summaries': ['#Person2# has trouble breathing. The doctor asks #Person2# about it and will send #Person2# to a pulmonary specialist.',
  '#Person1# invites Jimmy to go workout and persuades him into working out on arms and stomach.',
  "#Person1# plans to stop eating unhealthy foods, and #Person2# shares #Person2#'s healthy recipe with #Person1#.",
  "#Perso

# Model

In [29]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)
model=AutoModelForCausalLM.from_pretrained(cp,quantization_config=quantization_config)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

2026-02-19 01:52:36.470460: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771465956.651960      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771465956.706690      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771465957.151229      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771465957.151273      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771465957.151277      24 computation_placer.cc:177] computation placer alr

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [30]:
from peft import PeftModel

model = PeftModel.from_pretrained(
    model,
    "Pankaj121212/phi-2-cus",
    torch_dtype=torch.float16,is_trainable=False
)

adapter_config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/62.9M [00:00<?, ?B/s]

In [31]:
model.eval()
batch1['inputs']={k:v.to(device) for k,v in batch1['inputs'].items()}

In [32]:
%%time
# dont use max tokens by that model forcefully generate that many tokens which contain just repeated sentence
output=model.generate(
        **batch1['inputs'],
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,use_cache = True
    )

CPU times: user 6.03 s, sys: 1.25 s, total: 7.28 s
Wall time: 7.95 s


In [33]:
output

tensor([[50256, 50256, 50256,  ...,   284,   766,   257],
        [50256, 50256, 50256,  ...,  7405,   389, 19597],
        [50256, 50256, 50256,  ...,    17,     2, 25365],
        ...,
        [50256, 50256, 50256,  ...,  6834,   340,   338],
        [50256, 50256, 50256,  ...,   198,     2, 15439],
        [21106,   318,   281,  ...,   866,   262, 10481]], device='cuda:0')

In [34]:
output=tokenizer.batch_decode(output,skip_special_tokens=True)
output

["Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n    \n### Instruct: Summarise the following Conversation.\n\n#Person1#: Hello, how are you doing today?\n#Person2#: I'Ve been having trouble breathing lately.\n#Person1#: Have you had any type of cold lately?\n#Person2#: No, I haven't had a cold. I just have a heavy feeling in my chest when I try to breathe.\n#Person1#: Do you have any allergies that you know of?\n#Person2#: No, I don't have any allergies that I know of.\n#Person1#: Does this happen all the time or mostly when you are active?\n#Person2#: It happens a lot when I work out.\n#Person1#: I am going to send you to a pulmonary specialist who can run tests on you for asthma.\n#Person2#: Thank you for your help, doctor.\n\n### Output:\n#Person2# has trouble breathing and #Person1# recommends #Person2# to see a",
 "Below is an instruction that describes a task, paired wi

In [35]:
tokenizer.eos_token

'<|endoftext|>'

In [36]:
def generate_prediction(batch):
    inputs = {k: v.to(device) for k, v in batch['inputs'].items()}
    
    # Actually GENERATE text autoregressively
    outputs = model.generate(
        **inputs,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,use_cache = True
    )
    
    # Decode only the generated part (skip input tokens)
    generated = outputs[:, inputs['input_ids'].shape[1]:]
    return tokenizer.batch_decode(generated, skip_special_tokens=True)

In [37]:
# model.eval()
candidates=[]
references=[]
with torch.no_grad():
    for batch in tqdm(test_data):
        out=generate_prediction(batch)
        candidates+=out
        references+=batch['summaries']
        assert len(candidates)==len(references)

100%|██████████| 63/63 [08:36<00:00,  8.20s/it]


In [38]:
import evaluate
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")
rg=rouge.compute(predictions=candidates,references=[[r] for r in references])
bl=bleu.compute(predictions=candidates,references=references)
rg,bl

({'rouge1': np.float64(0.4334860363595153),
  'rouge2': np.float64(0.2095050199059973),
  'rougeL': np.float64(0.37703245422833476),
  'rougeLsum': np.float64(0.37704987174472226)},
 {'bleu': 0.17925207902387721,
  'precisions': [0.6606890042472865,
   0.43926288078224895,
   0.3027547472586253,
   0.18569995701389885],
  'brevity_penalty': 0.5015429315813837,
  'length_ratio': 0.5916928446771379,
  'translation_length': 8476,
  'reference_length': 14325})